In [1]:
import numpy as np
import torch
import pandas as pd
from torch import nn
# import dgl.function as fn
import torch.nn.functional as F
import gc
import os
import random
from tqdm import tqdm
from sklearn.metrics import auc,roc_auc_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
# from deepctr_torch.models.deepfm import FM,DNN
# from deepctr_torch.layers  import CIN,InteractingLayer,CrossNet,CrossNetMix
# from deepctr_torch.models.basemodel import *
from collections import defaultdict
from torch.optim import Optimizer
# import torchtext

In [2]:
import numpy as np
import torch
import pandas as pd
from torch import nn
import torch.nn.functional as F
import gc
import os
import random
from tqdm import tqdm
from sklearn.metrics import auc,roc_auc_score
from deepctr_torch.models.deepfm import FM,DNN
from deepctr_torch.layers  import CIN,InteractingLayer,CrossNet,CrossNetMix
from deepctr_torch.models.basemodel import *
from collections import defaultdict
from torch.optim import Optimizer
import torchtext
def reduce_mem(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('{:.2f} Mb, {:.2f} Mb ({:.2f} %)'.format(start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    gc.collect()
    return df

In [3]:
test_a=pd.read_csv('./wbdc2021/data/wedata/wechat_algo_data2/test_a.csv')

In [4]:
reduce_mem(test_a)

97.32 Mb, 36.50 Mb (62.50 %)


,userid,feedid,device
0,175282,50458,2
1,80036,42329,2
2,145791,85242,2
3,28430,9425,1
4,44393,11866,2
...,...,...,...
4252092,153322,51633,2
4252093,39430,20147,2
4252094,2524,89043,2
4252095,69629,27238,2


Please check the latest version manually on https://pypi.org/project/deepctr-torch/#history


In [8]:
import os
# os.environ["CUDA_DEVICE_ORDER"]='PCI_BUS_ID'
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [5]:
ratings=pd.read_csv('./wbdc2021/data/wedata/wechat_algo_data2/user_action.csv')
test_a=pd.read_csv('./wbdc2021/data/wedata/wechat_algo_data2/test_a.csv')
# test_b=pd.read_csv('./wbdc2021/data/wedata/wechat_algo_data2/test_b.csv')
# test_a['date_']=15
# # test_b['date_']=15
# feed_info=pd.read_csv('./wbdc2021/data/wedata/wechat_algo_data2/feed_info.csv')

In [1]:
ACTION_LIST = ["read_comment","like", "click_avatar", "forward",'comment','follow','favorite']#
PREDICT_LIST=["read_comment","like", "click_avatar", "forward",'comment','follow','favorite']

In [11]:
user_info=ratings.drop_duplicates('userid','first')[['userid','device']]

In [12]:
# 
print(ratings.shape)
ratings=ratings.drop_duplicates(['userid','feedid'],'last')
print(ratings.shape)
#ratings=ratings[(ratings.userid.isin(test_a.userid.unique()))]#|(ratings.feedid.isin(test_a.feedid.unique()))
print(ratings.shape)
## 建立 user _item 的对应表
userid_list=ratings.userid.unique().tolist()
userid2nid=dict(zip(userid_list,range(len(userid_list))))
num_user=len(userid_list)
print('user nums',num_user)

feedid_list=feed_info.feedid.unique().tolist()
num_feed=len(feedid_list)
print('feed nums',num_feed)

num_node_all=num_user+num_feed
print('all node nums',num_node_all)

feedid2nid=dict(zip(feedid_list,range(num_feed)))

(73175511, 13)
(71978260, 13)
(71978260, 13)
user nums 199999
feed nums 106444
all node nums 306443


In [1]:
# 这里就是要做整体的特征了
## 这里统一先用deedid 然后将feedid作为key来做映射
# ratings=pd.concat([ratings,test_a],axis=0)
def reduce_mem(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('{:.2f} Mb, {:.2f} Mb ({:.2f} %)'.format(start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    gc.collect()
    return df
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior  
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)


# # Target encode ps_car_11_cat
# trn, sub = target_encode(trn_df["ps_car_11_cat"], 
#                          sub_df["ps_car_11_cat"], 
#                          target=trn_df.target, 
#                          min_samples_leaf=100,
#                          smoothing=10,
#                          noise_level=0.01)


In [8]:
import pandas as pd
import numpy as np
# a=pd.read_pickle('./wbdc2021-semi/data/tmp/ratings_feat_df.pkl')
a=reduce_mem(a)

17655.76 Mb, 17237.04 Mb (2.37 %)


NameError: name 'gc' is not defined

In [9]:
# ratings=reduce_mem(ratings)
ratings,test_a=target_encode(ratings["userid"], 
                         test_a["userid"], 
                         target=ratings.read_comment, 
                         min_samples_leaf=100,
                         smoothing=10,
                         noise_level=0.001)

In [17]:
a=ratings.to_frame()

In [15]:
feed_embedding=pd.read_csv('./wbdc2021/data/wedata/wechat_algo_data2/feed_embeddings.csv')
feed_embedding['node']=feed_embedding['feedid'].apply(lambda x:feedid2nid[x])
feed_matrix=np.vstack(feed_embedding['feed_embedding'].apply(lambda x:np.array(list(map(float,x.strip().split(' '))))).values)

feed_emb=np.zeros(feed_matrix.shape)
indexs=feed_embedding['node'].values
for i in range(feed_matrix.shape[0]):
    feed_emb[indexs[i]]=feed_matrix[i]
del feed_embedding,feed_matrix
gc.collect()

22

In [10]:
# np.save('./tmp/knn_feed.npy',y_pred)

In [26]:
def machine_tag(ma):
    s=sorted(list(map(lambda x :x.split(' '),ma.split(';'))),key=lambda x:float(x[1]))
    s=[i[0] for i in s[-2:]]
    return '_'.join(s)

feed_info['machine_tag_list']=feed_info['machine_tag_list'].fillna('0 0')
tmp=feed_info['machine_tag_list'].astype('str').apply(machine_tag)
feed_info['machine_tag_id1']=tmp.apply(lambda x:x.split('_')[0])
#feed_info['machine_tag_id2']=tmp.apply(lambda x:x.split('_')[1] if len(x.split('_'))>1 else x.split('_')[0])

In [27]:
def manual_keyword(ma):
    s=ma.split(';')
    return '_'.join(s)

feed_info['manual_keyword_list']=feed_info['manual_keyword_list'].fillna('0;0')

tmp=feed_info['manual_keyword_list'].astype('str').apply( manual_keyword)
feed_info['manual_keyword_id1']=tmp.apply(lambda x:x.split('_')[0])
#feed_info['manual_keyword_id2']=tmp.apply(lambda x:x.split('_')[1] if len(x.split('_'))>1 else x.split('_')[0])

In [28]:
def manual_tag(ma):
    s=ma.split(';')
    return '_'.join(s)
feed_info['manual_tag_list']=feed_info['manual_keyword_list'].fillna('0;0')
tmp=feed_info['manual_tag_list'].astype('str').apply( manual_tag)
feed_info['manual_tag_id1']=tmp.apply(lambda x:x.split('_')[0])
#feed_info['manual_tag_id2']=tmp.apply(lambda x:x.split('_')[1] if len(x.split('_'))>1 else x.split('_')[0])


feed_info['machine_keyword_list']=feed_info['machine_keyword_list'].fillna('0;0')
tmp=feed_info['machine_keyword_list'].astype('str').apply( manual_tag)
feed_info['machine_keyword_id1']=tmp.apply(lambda x:x.split('_')[0])
#feed_info['machine_keyword_id2']=tmp.apply(lambda x:x.split('_')[1] if len(x.split('_'))>1 else x.split('_')[0])


In [29]:
feed_info['knn_feed']=np.load('./tmp/knn_2550_feed.npy')

In [15]:
# for dd in ['manual_keyword_id1','manual_tag_id1','machine_keyword_id1','manual_keyword_id2','manual_tag_id2','machine_keyword_id2','machine_tag_id1','machine_tag_id2','knn_feed']:
#     t=feed_info[feed_info['feedid'].isin(train_ratings.feedid)][dd]
#     v=feed_info[feed_info['feedid'].isin(val_ratings.feedid)][dd]
#     print(dd,v[~v.isin(t)].nunique())

In [16]:
np.random.seed(0)
class HyperParam(object):
    def __init__(self, alpha, beta):
        self.alpha = alpha
        self.beta = beta

    def sample_from_beta(self, alpha, beta, num, imp_upperbound):
        sample = np.random.beta(alpha, beta, num)   #贝塔分布
        I = []
        C = []
        for click_ratio in sample:   #Beta分布生成的click_ratio
            imp = random.random() * imp_upperbound
            #imp = imp_upperbound
            click = imp * click_ratio
            I.append(imp)
            C.append(click)
        return I, C

    def update_from_data_by_FPI(self, tries, success, iter_num, epsilon):
        '''用不动点迭代法 更新Beta里的参数 alpha和beta'''
        for i in range(iter_num):
            new_alpha, new_beta = self.__fixed_point_iteration(tries, success, self.alpha, self.beta)
            if abs(new_alpha-self.alpha)<epsilon and abs(new_beta-self.beta)<epsilon:
                break
            self.alpha = new_alpha
            self.beta = new_beta

    def __fixed_point_iteration(self, tries, success, alpha, beta):
        '''fixed point iteration 不动点迭代x_i+1 = g(x_i)'''
        sumfenzialpha = 0.0
        sumfenzibeta = 0.0
        sumfenmu = 0.0
        for i in range(len(tries)):
            #special.digamma(z)是 在z处取gamma函数值 再求log
            sumfenzialpha += (special.digamma(success[i]+alpha) - special.digamma(alpha))
            sumfenzibeta += (special.digamma(tries[i]-success[i]+beta) - special.digamma(beta))
            sumfenmu += (special.digamma(tries[i]+alpha+beta) - special.digamma(alpha+beta))
        return alpha*(sumfenzialpha/sumfenmu), beta*(sumfenzibeta/sumfenmu)

    def update_from_data_by_moment(self, tries, success):   #tries是各组总样本数，success是各组click=1的样本和
        '''用矩估计 更新Beta里的参数 alpha和beta'''
        mean, var = self.__compute_moment(tries, success)
        #print 'mean and variance: ', mean, var
        #self.alpha = mean * (mean*(1-mean)/(var+0.000001) - 1)
        self.alpha = (mean+0.000001) * ((mean+0.000001) * (1.000001 - mean) / (var+0.000001) - 1)
        #self.beta = (1-mean) * (mean*(1-mean)/(var+0.000001) - 1)
        self.beta = (1.000001 - mean) * ((mean+0.000001) * (1.000001 - mean) / (var+0.000001) - 1)

    def __compute_moment(self, tries, success):
        '''矩估计'''
        ctr_list = []
        var = 0.0
        for i in range(len(tries)):
            ctr_list.append(float(success[i])/(tries[i] + 0.000000001))
        mean = sum(ctr_list)/len(ctr_list)
        for ctr in ctr_list:
            var += pow(ctr-mean, 2)   #方差

        return mean, var/(len(ctr_list)-1)


In [15]:
# # 目标编码
# train_ratings=train_ratings.merge(feed_info[['feedid','authorid','bgm_song_id','bgm_singer_id','manual_tag_id1',
#                             'manual_tag_id2','machine_tag_id1','machine_tag_id2']],on='feedid',how='left')
# for col in ['authorid','bgm_song_id','bgm_singer_id','manual_tag_id1','manual_tag_id2','machine_tag_id1','machine_tag_id2']:
#     # target非1即0，对类别分组后把target全加起来也就是1的次数；统计分组后的数目即是该类别样本数目
#     target=train_ratings.groupby(col)['read_comment'].mean().reset_index()
#     target.columns=[col,col+'_tgcode']
#     # 上一步得到了映射的字典，据此将类别替换为目标编码后的值
#     feed_info= feed_info.merge(target,on=col,how='left')
#     feed_info[col+'_tgcode']=feed_info[col+'_tgcode'].fillna(0)
#     del train_ratings[col]
# gc.collect()

In [20]:
from sklearn.feature_extraction import FeatureHasher
hashing = FeatureHasher(n_features=16,input_type='string')
dense_arry1=hashing.transform(feed_info['manual_tag_list'].astype('str').values).toarray()
hashing = FeatureHasher(n_features=16,input_type='string')
dense_arry2=hashing.transform(feed_info['machine_tag_list'].astype('str').values).toarray()
hashing = FeatureHasher(n_features=8,input_type='string')
dense_arry3=hashing.transform(feed_info['machine_keyword_list'].astype('str').values).toarray()
hashing = FeatureHasher(n_features=8,input_type='string')
dense_arry4=hashing.transform(feed_info['manual_keyword_list'].astype('str').values).toarray()
hashing = FeatureHasher(n_features=16,input_type='string')
dense_arry5=hashing.transform(feed_info['description_char'].astype('str').values).toarray()

In [17]:
# train_ratings=train_ratings.merge(feed_info[['feedid','manual_tag_list']],on='feedid',how='left')
# tmp=train_ratings.groupby('userid')['manual_tag_list'].apply(lambda x:' '.join(x)).reset_index()

In [18]:
# del train_ratings['manual_tag_list']
# gc.collect()

In [18]:
max_day = 15
df = ratings#pd.concat([ratings, test_a], axis=0, ignore_index=True)
df = df.merge(feed_info[['feedid', 'authorid', 'videoplayseconds','bgm_song_id','manual_keyword_id1',]], on='feedid', how='left')
## 视频时长是秒，转换成毫秒，才能与play、stay做运算
df['videoplayseconds'] *= 1000
## 是否观看完视频（其实不用严格按大于关系，也可以按比例，比如观看比例超过0.9就算看完）
df[df['play']>240000]=240000
df['is_finish'] = (df['play'] >= df['videoplayseconds']).astype('int8')

# df['play_times'] = (df['play'] / df['videoplayseconds']).astype('float16')

play_cols = [
    'is_finish'
]
df=reduce_mem(df)
del ratings,test_a
gc.collect()

0

In [19]:
## 统计历史5天的曝光、转化、视频观看等情况（此处的转化率统计其实就是target encoding）
n_day =12
for stat_cols in tqdm([  ['userid'],['feedid'],['authorid'], ['userid', 'authorid'],['userid', 'bgm_song_id'],
        ['userid','manual_keyword_id1']]):
    f = '_'.join(stat_cols)
    stat_df = pd.DataFrame()
    for target_day in range(2, max_day + 1):
        left, right = max(target_day - n_day, 1), target_day - 1

        tmp = df[((df['date_'] >= left) & (df['date_'] <= right))].reset_index(drop=True)

        tmp['date_'] = target_day

        tmp['{}_{}day_count'.format(f, n_day)] = tmp.groupby(stat_cols)['date_'].transform('count')

        g = tmp.groupby(stat_cols)

        tmp['{}_{}day_finish_rate'.format(f, n_day)] = g[play_cols[0]].transform('mean')

        feats = ['{}_{}day_count'.format(f, n_day), '{}_{}day_finish_rate'.format(f, n_day)]

        # 这里是对播放进行的统计 但是我觉得无用因为test无法体现
#         for x in play_cols[1:]:
#             for stat in ['max', 'mean']:
#                 tmp['{}_{}day_{}_{}'.format(f, n_day, x, stat)] = g[x].transform(stat)

#                 feats.append('{}_{}day_{}_{}'.format(f, n_day, x, stat))

        # 这部分类似目标编码了
        for y in PREDICT_LIST:
            tmp['{}_{}day_{}_sum'.format(f, n_day, y)] = g[y].transform('sum')

            tmp['{}_{}day_{}_mean'.format(f, n_day, y)] = g[y].transform('mean')
            feats.extend(['{}_{}day_{}_sum'.format(f, n_day, y), '{}_{}day_{}_mean'.format(f, n_day, y)])
        tmp = tmp[stat_cols + feats + ['date_']].drop_duplicates(stat_cols + ['date_']).reset_index(drop=True)
        tmp.to_pickle('./tmp/{}_feat_{}.pkl'.format(target_day,'_'.join(stat_cols)))
        #stat_df = pd.concat([stat_df, tmp], axis=0, ignore_index=True)
        del g, tmp
    #df = df.merge(stat_df, on=stat_cols + ['date_'], how='left')

    #del stat_df

    gc.collect()

# for f1, f2 in tqdm([['userid', 'authorid']]):
#     tmp=df.groupby([f1, f2])
#     for y in PREDICT_LIST:
#         df['{}_{}_{}_mean'.format(f1, f2,y)] = tmp[y].transform('mean')


100%|██████████| 6/6 [57:56<00:00, 579.39s/it]


In [1]:
import pandas as pd

In [2]:
# tmp=pd.read_pickle('./tmp/3_feat_userid_authorid.pkl')

In [7]:
tmp

,userid,authorid,userid_authorid_12day_count,userid_authorid_12day_finish_rate,userid_authorid_12day_read_comment_sum,userid_authorid_12day_read_comment_mean,userid_authorid_12day_like_sum,userid_authorid_12day_like_mean,userid_authorid_12day_click_avatar_sum,userid_authorid_12day_click_avatar_mean,userid_authorid_12day_forward_sum,userid_authorid_12day_forward_mean,userid_authorid_12day_comment_sum,userid_authorid_12day_comment_mean,userid_authorid_12day_follow_sum,userid_authorid_12day_follow_mean,userid_authorid_12day_favorite_sum,userid_authorid_12day_favorite_mean,date_
0,0,4837,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,0,2378,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2,0,10512,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,0,15696,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4,0,4845,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7983620,250246,711,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
7983621,250246,14473,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
7983622,250248,18635,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
7983623,250248,4996,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [21]:
import pickle

In [20]:
# m_feat=tmp.columns[2:]
# a=pd.DataFrame(tmp[m_feat].mean(0))
# a=a.fillna(-1)
# m_feat.drop('date_')
m_feat

Index(['userid_authorid_12day_count', 'userid_authorid_12day_finish_rate',
       'userid_authorid_12day_read_comment_sum',
       'userid_authorid_12day_read_comment_mean',
       'userid_authorid_12day_like_sum', 'userid_authorid_12day_like_mean',
       'userid_authorid_12day_click_avatar_sum',
       'userid_authorid_12day_click_avatar_mean',
       'userid_authorid_12day_forward_sum',
       'userid_authorid_12day_forward_mean',
       'userid_authorid_12day_comment_sum',
       'userid_authorid_12day_comment_mean',
       'userid_authorid_12day_follow_sum', 'userid_authorid_12day_follow_mean',
       'userid_authorid_12day_favorite_sum',
       'userid_authorid_12day_favorite_mean', 'date_'],
      dtype='object')

{'userid_12day_count': 191.42442270249902,
 'userid_12day_finish_rate': 0.0,
 'userid_12day_read_comment_sum': -1.0,
 'userid_12day_read_comment_mean': -1.0,
 'userid_12day_like_sum': -1.0,
 'userid_12day_like_mean': -1.0,
 'userid_12day_click_avatar_sum': -1.0,
 'userid_12day_click_avatar_mean': 0.0,
 'userid_12day_forward_sum': -1.0,
 'userid_12day_forward_mean': 0.0,
 'userid_12day_comment_sum': -1.0,
 'userid_12day_comment_mean': 0.0,
 'userid_12day_follow_sum': -1.0,
 'userid_12day_follow_mean': 0.0,
 'userid_12day_favorite_sum': -1.0,
 'userid_12day_favorite_mean': 0.0}

In [36]:
max_day = 14
# feed_info=pd.read_csv('./wbdc2021-semi/data/wedata/wechat_algo_data2/feed_info.csv')
# df = pd.concat([ratings, test_a], axis=0, ignore_index=True)
# df=pd.read_csv('./wbdc2021-semi/data/wedata/wechat_algo_data2/user_action.csv')#pd.concat([pd.read_pickle('./tmp/sample_df.pkl'),test_a],axis=0,ignore_index=True)
# df = df.merge(feed_info[['feedid', 'authorid', 'videoplayseconds','bgm_song_id','manual_keyword_id1']], on='feedid', how='left')
# ## 视频时长是秒，转换成毫秒，才能与play、stay做运算
# df['videoplayseconds'] *= 1000
## 是否观看完视频（其实不用严格按大于关系，也可以按比例，比如观看比例超过0.9就算看完）

# df=reduce_mem(df)
# del ratings,test_a
gc.collect()

for stat_cols in tqdm([  ['userid'],['feedid'],['authorid'], ['userid', 'authorid'],['userid', 'bgm_song_id'],
        ['userid','manual_keyword_id1']]):
    f = '_'.join(stat_cols)
    stat_df = pd.DataFrame()
    for target_day in range(2, max_day + 1):
#         tmp.to_pickle('./tmp/{}_feat_{}.pkl'.format(target_day,'_'.join(stat_cols)))
        tmp=pd.read_pickle('./tmp/{}_feat_{}.pkl'.format(target_day,'_'.join(stat_cols)))
        tmp=reduce_mem(tmp)
        stat_df = pd.concat([stat_df, tmp], axis=0, ignore_index=True)
        del tmp
    tmp_feat=stat_df.columns[len(stat_cols):]
    tmp_feat=tmp_feat.drop('date_')
    mean_tmp=stat_df[tmp_feat].mean(0)
    mean_tmp.fillna(-1,inplace=True)
    mean_tmp=mean_tmp.to_dict()
    pickle.dump(mean_tmp,open('./tmp/{}_feat_mean.pkl'.format('_'.join(stat_cols)),'wb'))
    df = df.merge(stat_df, on=stat_cols + ['date_'], how='left')
    for kk,vv in mean_tmp.items():
        df[kk]=df[kk].fillna(vv)
    df=reduce_mem(df)
    del stat_df

    gc.collect()


  0%|          | 0/6 [00:00<?, ?it/s]

5.85 Mb, 3.87 Mb (33.93 %)
7.53 Mb, 4.97 Mb (33.93 %)
8.51 Mb, 5.63 Mb (33.93 %)
9.08 Mb, 6.00 Mb (33.93 %)
9.44 Mb, 6.24 Mb (33.93 %)
9.71 Mb, 6.41 Mb (33.93 %)
9.93 Mb, 6.56 Mb (33.93 %)
10.10 Mb, 6.67 Mb (33.93 %)
10.25 Mb, 6.77 Mb (33.93 %)
10.38 Mb, 6.86 Mb (33.93 %)
10.49 Mb, 6.93 Mb (33.93 %)
10.57 Mb, 6.98 Mb (33.93 %)
10.62 Mb, 7.02 Mb (33.93 %)


/opt/conda/envs/pytorch_py3/lib/python3.6/site-packages/numpy/core/_methods.py:47: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


6350.49 Mb, 5931.78 Mb (6.59 %)


 17%|█▋        | 1/6 [01:19<06:37, 79.55s/it]

...

 83%|████████▎ | 5/6 [29:37<07:55, 475.16s/it]

139.89 Mb, 98.36 Mb (29.69 %)
257.57 Mb, 181.10 Mb (29.69 %)
377.57 Mb, 265.48 Mb (29.69 %)
482.88 Mb, 339.52 Mb (29.69 %)
567.57 Mb, 399.07 Mb (29.69 %)
652.37 Mb, 458.70 Mb (29.69 %)
740.65 Mb, 520.77 Mb (29.69 %)
829.99 Mb, 583.59 Mb (29.69 %)
918.67 Mb, 645.94 Mb (29.69 %)
1018.66 Mb, 716.24 Mb (29.69 %)
1118.81 Mb, 786.66 Mb (29.69 %)
1201.65 Mb, 844.91 Mb (29.69 %)
1220.37 Mb, 858.07 Mb (29.69 %)


/opt/conda/envs/pytorch_py3/lib/python3.6/site-packages/numpy/core/_methods.py:47: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


17655.76 Mb, 17237.04 Mb (2.37 %)


100%|██████████| 6/6 [38:53<00:00, 388.97s/it]


In [40]:
df.shape

(73175511, 113)

In [21]:
max_day = 15
# df = pd.concat([ratings, test_a], axis=0, ignore_index=True)
df=pd.concat([pd.read_pickle('./tmp/sample_df.pkl'),test_a],axis=0,ignore_index=True)
df = df.merge(feed_info[['feedid', 'authorid', 'videoplayseconds','bgm_song_id','manual_keyword_id1']], on='feedid', how='left')
## 视频时长是秒，转换成毫秒，才能与play、stay做运算
df['videoplayseconds'] *= 1000
## 是否观看完视频（其实不用严格按大于关系，也可以按比例，比如观看比例超过0.9就算看完）

df=reduce_mem(df)
del ratings,test_a
gc.collect()

for stat_cols in tqdm([  ['userid'],['feedid'],['authorid'], ['userid', 'authorid'],['userid', 'bgm_song_id'],
        ['userid','manual_keyword_id1']]):
    f = '_'.join(stat_cols)
    stat_df = pd.DataFrame()
    for target_day in range(2, max_day + 1):
#         tmp.to_pickle('./tmp/{}_feat_{}.pkl'.format(target_day,'_'.join(stat_cols)))
        tmp=pd.read_pickle('./tmp/{}_feat_{}.pkl'.format(target_day,'_'.join(stat_cols)))
        tmp=reduce_mem(tmp)
        stat_df = pd.concat([stat_df, tmp], axis=0, ignore_index=True)
        del tmp
    df = df.merge(stat_df, on=stat_cols + ['date_'], how='left')
    df=reduce_mem(df)

    del stat_df

    gc.collect()


ERROR! Session/line number was not unique in database. History logging moved to new session 16
8648.02 Mb, 3363.12 Mb (61.11 %)


  0%|          | 0/6 [00:00<?, ?it/s]

5.85 Mb, 3.87 Mb (33.93 %)
7.53 Mb, 4.97 Mb (33.93 %)
8.51 Mb, 5.63 Mb (33.93 %)
9.08 Mb, 6.00 Mb (33.93 %)
9.44 Mb, 6.24 Mb (33.93 %)
9.71 Mb, 6.41 Mb (33.93 %)
9.93 Mb, 6.56 Mb (33.93 %)
10.10 Mb, 6.67 Mb (33.93 %)
10.25 Mb, 6.77 Mb (33.93 %)
10.38 Mb, 6.86 Mb (33.93 %)
10.49 Mb, 6.93 Mb (33.93 %)
10.57 Mb, 6.98 Mb (33.93 %)
10.62 Mb, 7.02 Mb (33.93 %)
10.67 Mb, 7.05 Mb (33.93 %)


 17%|█▋        | 1/6 [01:30<07:30, 90.04s/it]

5645.23 Mb, 5284.90 Mb (6.38 %)
1.50 Mb, 1.10 Mb (26.53 %)
1.81 Mb, 1.33 Mb (26.53 %)
2.09 Mb, 1.54 Mb (26.53 %)
2.34 Mb, 1.82 Mb (22.45 %)
2.63 Mb, 2.04 Mb (22.45 %)
2.95 Mb, 2.29 Mb (22.45 %)
3.25 Mb, 2.52 Mb (22.45 %)
3.55 Mb, 2.75 Mb (22.45 %)
3.78 Mb, 2.94 Mb (22.45 %)
3.99 Mb, 3.10 Mb (22.45 %)
4.18 Mb, 3.24 Mb (22.45 %)
4.38 Mb, 3.40 Mb (22.45 %)
4.56 Mb, 3.54 Mb (22.45 %)
4.68 Mb, 3.63 Mb (22.45 %)


...

 67%|██████▋   | 4/6 [19:46<10:54, 327.37s/it]

128.70 Mb, 86.54 Mb (32.76 %)
249.01 Mb, 167.44 Mb (32.76 %)
383.98 Mb, 258.20 Mb (32.76 %)
506.76 Mb, 340.76 Mb (32.76 %)
612.45 Mb, 411.82 Mb (32.76 %)
721.74 Mb, 485.31 Mb (32.76 %)
834.06 Mb, 560.83 Mb (32.76 %)
950.32 Mb, 639.01 Mb (32.76 %)
1068.54 Mb, 718.50 Mb (32.76 %)
1204.31 Mb, 809.79 Mb (32.76 %)
1346.82 Mb, 905.62 Mb (32.76 %)
1468.02 Mb, 987.12 Mb (32.76 %)
1491.47 Mb, 1002.89 Mb (32.76 %)
1529.82 Mb, 1028.67 Mb (32.76 %)
13092.14 Mb, 13092.14 Mb (0.00 %)


 83%|████████▎ | 5/6 [30:12<06:56, 416.77s/it]

139.89 Mb, 98.36 Mb (29.69 %)
257.57 Mb, 181.10 Mb (29.69 %)
377.57 Mb, 265.48 Mb (29.69 %)
482.88 Mb, 339.52 Mb (29.69 %)
567.57 Mb, 399.07 Mb (29.69 %)
652.37 Mb, 458.70 Mb (29.69 %)
740.65 Mb, 520.77 Mb (29.69 %)
829.99 Mb, 583.59 Mb (29.69 %)
918.67 Mb, 645.94 Mb (29.69 %)
1018.66 Mb, 716.24 Mb (29.69 %)
1118.81 Mb, 786.66 Mb (29.69 %)
1201.65 Mb, 844.91 Mb (29.69 %)
1220.37 Mb, 858.07 Mb (29.69 %)
1248.18 Mb, 877.63 Mb (29.69 %)
15374.25 Mb, 15013.92 Mb (2.34 %)


100%|██████████| 6/6 [40:05<00:00, 400.92s/it]


In [21]:
df.to_pickle('./tmp/feat_df_new.pkl')

In [2]:
# del ratings, test_a
# gc.collect()
df=pd.read_pickle('./tmp/feat_df_new.pkl')

In [41]:
feat=df.columns[21:]
print(len(feat))

92


In [43]:
# df.fillna(-1,inplace=True)
feat=df.columns[21:]
print(len(feat))
pickle.dump(feat,open('./wbdc2021-semi/data/tmp/feat_list.pkl','wb'))
# mms = MinMaxScaler(feature_range=(0, 1))
normolizer_dict={}
for f in tqdm(feat):
    tmp=df[f].values.astype('float16').clip(-1,1e8)
    tmp_max=tmp.max() # 这里 或许我得保留均值和方差
    tmp_min=tmp.min()
    normolizer_dict[f+'_max']=tmp_max
    normolizer_dict[f+'_min']=tmp_min
    df[f]=((tmp-tmp_min)/tmp_max).astype('float16')
# df[feat] = mms.fit_transform(df[feat].values.astype('float16').clip(-1,1e8))

92


100%|██████████| 92/92 [02:11<00:00,  1.43s/it]


In [44]:
df['reg']=np.sqrt((df['play']/df['videoplayseconds']).values)

In [46]:
df.to_pickle('./tmp/ratings_feat_df.pkl')
pickle.dump(normolizer_dict,open('./wbdc2021-semi/data/tmp/normolizer_dict.pkl','wb'))

In [27]:
# df.to_pickle('./tmp/feat_df.pkl')
# del ratings,test_a
gc.collect()
# df=pd.read_pickle('./tmp/feat_df.pkl')
# df=reduce_mem(df)
feat=df.columns[21:-1]
len(feat)

92

In [28]:
feed_data={}
user_data={}

### 给节点传入特征
feed_feats=['feedid','authorid','videoplayseconds','bgm_song_id','bgm_singer_id','description_char'
            ,'manual_keyword_id1','manual_tag_id1','machine_keyword_id1'
            ,'machine_tag_id1','knn_feed']
# , 'read_comment_rate', 'like_rate', 'click_avatar_rate',
#        'forward_rate', 'comment_rate', 'follow_rate', 'favorite_rate'
feeds=feed_info[feed_feats]

for f in feed_feats:
    feeds[f]=feeds[f].fillna(0)

# 对id 重新进行编码，类别特征
for f in ['bgm_song_id', 'bgm_singer_id','authorid','knn_feed']:
    gens=LabelEncoder()
    feeds[f]=gens.fit_transform(feeds[f].astype('str'))
    feed_data[f]=torch.from_numpy(feeds[f].values)

# 对于tag 和keyword 要联合编码
gens=LabelEncoder()
tmp=pd.concat([feeds[f].astype('str') for f in ['manual_tag_id1','machine_tag_id1'] ])
gens=gens.fit(tmp)
for f in ['manual_tag_id1','machine_tag_id1']:
    feeds[f]=gens.transform(feeds[f].astype('str'))
    feed_data[f]=torch.from_numpy(feeds[f].values)
    
gens=LabelEncoder()
tmp=pd.concat([feeds[f].astype('str') for f in ['manual_keyword_id1','machine_keyword_id1'] ])
gens=gens.fit(tmp)
for f in ['manual_keyword_id1','machine_keyword_id1']:
    feeds[f]=gens.transform(feeds[f].astype('str'))
    feed_data[f]=torch.from_numpy(feeds[f].values)
    
## 连续特征进行归一化
dense_features=['videoplayseconds']
for f in dense_features:
    feeds[f]=np.log(feeds[f] + 1.0)
mms = MinMaxScaler(feature_range=(0, 1))
feeds[dense_features] = mms.fit_transform(feeds[dense_features])
feed_data['dense']=torch.from_numpy(feeds[dense_features].values.astype('float32'))
# feed_data['manuual_tag_list_emb']=torch.from_numpy(seq).long()
feed_data['hash_dense']=torch.from_numpy(np.hstack([dense_arry1,dense_arry2,dense_arry4,dense_arry3,dense_arry5]).astype('float32'))
#------------------------------------------------------------------------------------------------
# user 
user_feats=['userid','device']
for f in user_feats:
    user_info[f]=user_info[f].fillna(0)
    
for f in ['device']:
    gens=LabelEncoder()
    user_info[f]=gens.fit_transform(user_info[f])
    user_data[f]=torch.from_numpy(user_info[f].values)   
# 传入userid
user_data['userid']=torch.from_numpy(user_info['userid'].apply(lambda x:userid2nid[x]).values)
#
# user_dense=['follow_rate','comment_rate','click_avatar_rate',
#                 'like_rate','read_comment_rate','favorite_rate']
# user_data['user_dense']=torch.from_numpy(user_info[user_dense].values.astype('float32'))

/home/tione/notebook/envs/tf1/lib/python3.6/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/tione/notebook/envs/tf1/lib/python3.6/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/tione/notebook/envs/tf1/lib/python3.6/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [29]:
item_texts={}
user_texts={}
for f in ['manual_tag_list','manual_keyword_list','machine_keyword_list','asr','description','ocr']:#ocr
    feed_info[f]=feed_info[f].astype('str').apply(lambda x:x.replace(';',' '))
    item_texts[f]=feed_info[f].values
# for f in ['manual_tag_user_read_comment_list']:
#     user_texts[f]=user_info[f].astype('str').values

In [30]:
import torchtext
class BagOfWordsPretrained(nn.Module):
    def __init__(self, field, hidden_dims):
        super().__init__()

        input_dims = field.vocab.vectors.shape[1]
        self.emb = nn.Embedding(
            len(field.vocab.itos), input_dims,
            padding_idx=field.vocab.stoi[field.pad_token])
        self.emb.weight[:] = field.vocab.vectors
        self.proj = nn.Linear(input_dims, hidden_dims)
        nn.init.xavier_uniform_(self.proj.weight)
        nn.init.constant_(self.proj.bias, 0)

        disable_grad(self.emb) # 词向量不可训练

    def forward(self, x):
        """
        x: (batch_size, max_length) LongTensor
        length: (batch_size,) LongTensor
        """
        x = self.emb(x).sum(1)# / length.unsqueeze(1).float() # 归一化
        return self.proj(x)

class BagOfWords(nn.Module):
    def __init__(self, field, hidden_dims):
        super().__init__()
        self.att_emb=Attn(hidden_dims)
        self.emb = nn.Embedding(
            len(field.vocab.itos), hidden_dims,
            padding_idx=field.vocab.stoi[field.pad_token])
        nn.init.xavier_uniform_(self.emb.weight)

    def forward(self, x):
        return self.att_emb(self.emb(x))#.mean(1)#/ length.unsqueeze(1).float() # 归一化
class text_emb(nn.Module):
    def __init__(self,weight):
        super().__init__()
        self.att_emb=Attn(weight.shape[1])
        self.emb = nn.Embedding(
            weight.shape[0],weight.shape[1],
            padding_idx=0)
#         nn.init.xavier_uniform_(self.emb.weight)
        self.emb.weight.data.copy_(torch.from_numpy(weight).float())
#         self.emb.requires_grad_=False
    def forward(self, x):
        return self.att_emb(self.emb(x))#.mean(1)#/ length.unsqueeze(1).float() # 归一化
# if textset is not None:
#         for column, field in textset.fields.items():
#             if field.vocab.vectors:
#                 module_dict[column] = BagOfWordsPretrained(field, hidden_dims)
#             else:
#                 module_dict[column] = BagOfWords(field, hidden_dims)

tokenize = lambda x: x.split(' ')
fields = {}
examples = []
for key, texts in item_texts.items():
    if  key in ['ocr','asr','description']:
        fields[key] = torchtext.data.Field(include_lengths=True, lower=True,tokenize=tokenize, batch_first=True, fix_length=64)
    else:
        fields[key] = torchtext.data.Field(include_lengths=True, lower=True,tokenize=tokenize, batch_first=True, fix_length=5)
    
for i in range(len(feed_info)):
    example = torchtext.data.Example.fromlist(
        [item_texts[key][i] for key in item_texts.keys()],
        [(key, fields[key]) for key in item_texts.keys()])  #( [feat1,feat2], [(key1,field1),(key2,field2)] )
    examples.append(example)
textset = torchtext.data.Dataset(examples, fields)
for key, field in fields.items():
    field.build_vocab(getattr(textset, key))
    
# user text
# fields = {}
# examples = []
# user_fields={}
# for key, texts in user_texts.items():
#     if 'user' in key:
#         fields[key] = torchtext.data.Field(include_lengths=True, lower=True,tokenize=tokenize, batch_first=True, fix_length=64)
# user_fields['manual_tag_user_read_comment_list']=fields['manual_tag_list']
    
# for i in range(len(user_info)):
#     example = torchtext.data.Example.fromlist(
#         [user_texts[key][i] for key in user_texts.keys()],
#         [(key, user_fields[key]) for key in user_texts.keys()])  #( [feat1,feat2], [(key1,field1),(key2,field2)] )
#     examples.append(example)
    
# userset = torchtext.data.Dataset(examples, user_fields)
# for key, field in user_fields.items():
#     field.build_vocab(getattr(userset, key))

/home/tione/notebook/envs/tf1/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/tione/notebook/envs/tf1/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [31]:
for field_name, field in textset.fields.items():
    examples = [getattr(textset[i], field_name) for i in range(len(feed_info))]

    tokens, lengths = field.process(examples)

    if not field.batch_first:
        tokens = tokens.t()
    # 给feed +上文本向量
    feed_data[field_name] = tokens

# for field_name, field in userset.fields.items():
#     examples = [getattr(userset[i], field_name) for i in range(len(user_info))]

#     tokens, lengths = field.process(examples)

#     if not field.batch_first:
#         tokens = tokens.t()
#     # 给feed +上文本向量
#     user_data[field_name] = tokens

In [ ]:


tokenize = lambda x: x.split(' ')
fields = {}
examples = []
for key, texts in item_texts.items():
    if  key in ['ocr','asr','description']:
        fields[key] = torchtext.data.Field(include_lengths=True, lower=True,tokenize=tokenize, batch_first=True, fix_length=64)
    else:
        fields[key] = torchtext.data.Field(include_lengths=True, lower=True,tokenize=tokenize, batch_first=True, fix_length=5)
    
for i in range(len(feed_info)):
    example = torchtext.data.Example.fromlist(
        [item_texts[key][i] for key in item_texts.keys()],
        [(key, fields[key]) for key in item_texts.keys()])  #( [feat1,feat2], [(key1,field1),(key2,field2)] )
    examples.append(example)
textset = torchtext.data.Dataset(examples, fields)
for key, field in fields.items():
    field.build_vocab(getattr(textset, key))
    
for field_name, field in textset.fields.items():
    examples = [getattr(textset[i], field_name) for i in range(len(feed_info))]

    tokens, lengths = field.process(examples)

    if not field.batch_first:
        tokens = tokens.t()
    # 给feed +上文本向量
    feed_data[field_name] = tokens

In [32]:
class Model(nn.Module):
    def __init__(self,usr_data,feed_data,feed_embed,graph_emb):
        super().__init__()
        self.feed_data=feed_data
        self.user_data=user_data
        user_dict={'device':2,'userid':128}
        feed_dict={'bgm_song_id':16, 'bgm_singer_id':16,'authorid':16,'dense':32,'hash_dense':32
       ,'manual_keyword_id1':16,'manual_tag_id1':16,'machine_keyword_id1':16
            ,'machine_tag_id1':16,'knn_feed':16,
           'manual_tag_list':32,'manual_keyword_list':32,'machine_keyword_list':32,'asr':32,'description':32,'ocr':32
                  }
        self.model_dict=_init_input_modules(user_data,feed_data, user_dict,feed_dict)
        self.spare_liner=nn.Linear(8*16,128)
        self.dense_liner=nn.Linear(32*2,128)
        self.text_liner=nn.Linear(32*6+512+64,128)
        self.feed_embed= nn.Parameter(torch.from_numpy(feed_embed).float(),requires_grad=False)
        self.graph= nn.Parameter(torch.from_numpy(graph_emb).float(),requires_grad=False)
        self.reg_liner=nn.Linear(128,1)
        self.dynami_dense=nn.Linear(92,64)
        self.cross1=CrossNetMix(sum(user_dict.values())+128*3+64,layer_num=4)
        self.cross2=CrossNetMix(sum(user_dict.values())+128*3+64,layer_num=4)
        self.cross3=CrossNetMix(sum(user_dict.values())+128*3+64,layer_num=4)
        self.cross4=CrossNetMix(sum(user_dict.values())+128*3+64,layer_num=4)
        self.cross5=CrossNetMix(sum(user_dict.values())+128*3+64,layer_num=4)
        self.cross6=CrossNetMix(sum(user_dict.values())+128*3+64,layer_num=4)
        self.cross7=CrossNetMix(sum(user_dict.values())+128*3+64,layer_num=4)
#         self.dnn=DNN(sum(user_dict.values())+128*3+64,(128,128),dropout_rate=0.1)
        self.mmoe=MMOELayer(sum(user_dict.values())+128*3+64, mmoe_hidden_dim=128,num_task=6,n_expert=5,expert_activation=None)
#         self.att1=Attn(sum(user_dict.values())+128*3+64)
#         self.att2=Attn(sum(user_dict.values())+128*3+64)
#         self.att3=Attn(sum(user_dict.values())+128*3+64)
#         self.att4=Attn(sum(user_dict.values())+128*3+64)
        
        self.liner1=nn.Linear(128+sum(user_dict.values())+128*3+64,1)
        self.liner2=nn.Linear(128+sum(user_dict.values())+128*3+64,1)
        self.liner3=nn.Linear(128+sum(user_dict.values())+128*3+64,1)
        self.liner4=nn.Linear(128+sum(user_dict.values())+128*3+64,1)
        self.liner5=nn.Linear(128+sum(user_dict.values())+128*3+64,1)
        self.liner6=nn.Linear(128+sum(user_dict.values())+128*3+64,1)
        self.liner7=nn.Linear(128+sum(user_dict.values())+128*3+64,1)
    def forward(self,userid,feedid,batch_dense,is_train=True):
        user_projections=[]
#         feed_projections=[]
        dense_embedding=[]
        sparse_embedding=[]
        text_embedding=[]
        for feature, data in self.user_data.items():
            module = self.model_dict[feature]
            result = module(data)
            user_projections.append(result)
        for feature, data in self.feed_data.items():
#             print(feature)
            module = self.model_dict[feature]
            result = module(data)
            if result.shape[-1]==16:
                sparse_embedding.append(result)
            elif 'dense' in feature:
                dense_embedding.append(result)
            else:
#                 print(result.shape)
                text_embedding.append(result)
#         print(user_projections)

        user_feat=torch.cat(user_projections,-1)
        spare_emb=self.spare_liner(torch.cat(sparse_embedding,-1))
        dense_emb=self.dense_liner(torch.cat(dense_embedding,-1))
        text_emb=self.text_liner(torch.cat(text_embedding+[self.feed_embed,self.graph],-1))  
        feed_feat=torch.cat([spare_emb,dense_emb,text_emb],-1) #128*3
        dynami_dense=self.dynami_dense(batch_dense)
        combine=torch.cat([user_feat[userid],feed_feat[feedid],dynami_dense],axis=-1)
        cross1=self.cross1(combine)
        cross2=self.cross2(combine)
        cross3=self.cross3(combine)
        cross4=self.cross4(combine)
        cross5=self.cross5(combine)
        cross6=self.cross6(combine)
        cross7=self.cross7(combine)
        outs=self.mmoe(combine)

        
        logit_gnn1=self.liner1(torch.cat([outs[0],cross1],axis=-1))#+ffm1#128+1+128*2
        logit_gnn2=self.liner2(torch.cat([outs[1],cross2],axis=-1))#+ffm2
        
        logit_gnn3=self.liner3(torch.cat([outs[2],cross3],axis=-1))#+ffm3
        logit_gnn4=self.liner4(torch.cat([outs[3],cross4],axis=-1))#+ffm4
        logit_gnn5=self.liner3(torch.cat([outs[0],cross5],axis=-1))#+ffm3
        logit_gnn6=self.liner4(torch.cat([outs[2],cross6],axis=-1))#+ffm4
        logit_gnn7=self.liner4(torch.cat([outs[4],cross7],axis=-1))#+ffm4
        logit_reg=self.reg_liner(outs[5])
        return logit_gnn1,logit_gnn2,logit_gnn3,logit_gnn4,logit_gnn5,logit_gnn6,logit_gnn7,logit_reg
    
def _init_input_modules(user_data,feed_data, user_dict,feed_dict):
    # We initialize the linear projections of each input feature ``x`` as
    # follows:
    # * If ``x`` is a scalar integral feature, we assume that ``x`` is a categorical
    #   feature, and assume the range of ``x`` is 0..max(x).
    # * If ``x`` is a float one-dimensional feature, we assume that ``x`` is a
    #   numeric vector.
    # * If ``x`` is a field of a textset, we process it as bag of words.
    module_dict = nn.ModuleDict()
    for column, data in user_data.items():
        if column in user_texts.keys():
            continue
        if data.dtype == torch.float32: # 数值类型的特征
            assert data.ndim == 2
            m = nn.Linear(data.shape[1],user_dict[column]) # 数值特征 做个线性变换
            nn.init.xavier_uniform_(m.weight)
            nn.init.constant_(m.bias, 0)
        elif data.dtype == torch.int64:
            assert data.ndim == 1  # 整形的单值特征做个embedding
            m = nn.Embedding(data.max() + 2, user_dict[column], padding_idx=-1)
            nn.init.xavier_uniform_(m.weight)
        module_dict[column] = m  # 不同的特征名字对应不同的处理moderl 这里或许可以加FM进去
    
    for column, data in feed_data.items():
        if column in item_texts.keys():
            continue
        if column =='manuual_tag_list_emb':
            continue
        if data.dtype == torch.float32: # 数值类型的特征
            assert data.ndim == 2
            m = nn.Linear(data.shape[1],feed_dict[column]) # 数值特征 做个线性变换
            nn.init.xavier_uniform_(m.weight)
            nn.init.constant_(m.bias, 0)
        elif data.dtype == torch.int64:
            assert data.ndim == 1  # 整形的单值特征做个embedding
            m = nn.Embedding(data.max() + 2, feed_dict[column], padding_idx=-1)
            nn.init.xavier_uniform_(m.weight)
        module_dict[column] = m  # 不同的特征名字对应不同的处理moderl 这里或许可以加FM进去
        
    if textset is not None:
        for column, field in textset.fields.items():
            if field.vocab.vectors:
                module_dict[column] = BagOfWordsPretrained(field,feed_dict[column])
            else:
                module_dict[column] = BagOfWords(field,feed_dict[column])
#     if userset is not None:
#         for column, field in userset.fields.items():
#             if field.vocab.vectors:
#                 module_dict[column] = BagOfWordsPretrained(field,user_dict[column])
#             else:
#                 module_dict[column] = BagOfWords(field,user_dict[column])
#     module_dict['manual_tag_user_read_comment_list']=module_dict['manual_tag_list']
    return module_dict

class MMOELayer(nn.Module):
    def __init__(self, hidden_size, mmoe_hidden_dim=128,num_task=4,n_expert=3,expert_activation=None):
        super(MMOELayer, self).__init__()
         # experts
        self.num_task=num_task
        self.expert_activation = expert_activation
        self.experts = torch.nn.Parameter(torch.rand(hidden_size, mmoe_hidden_dim, n_expert).cuda(), requires_grad=True)
        self.experts.data.normal_(0, 1)
        self.experts_bias = torch.nn.Parameter(torch.rand(mmoe_hidden_dim, n_expert).cuda(), requires_grad=True)
        # gates
        self.gates = [torch.nn.Parameter(torch.rand(hidden_size, n_expert), requires_grad=True).cuda() for _ in range(num_task)]
        for gate in self.gates:
            gate.data.normal_(0, 1)
        self.gates_bias = [torch.nn.Parameter(torch.rand(n_expert), requires_grad=True).cuda() for _ in range(num_task)]
        for i in range(num_task):
            setattr(self, 'task_{}_dnn'.format(i+1),DNN(mmoe_hidden_dim,(128,128),dropout_rate=0.2,use_bn=True))
    def forward(self,x):
         # mmoe
        experts_out = torch.einsum('ij, jkl -> ikl', x, self.experts) # batch * mmoe_hidden_size * num_experts
        experts_out += self.experts_bias
        if self.expert_activation is not None:
            experts_out = self.expert_activation(experts_out)
        
        gates_out = list()
        for idx, gate in enumerate(self.gates):
            gate_out = torch.einsum('ab, bc -> ac',x, gate) # batch * num_experts
            if self.gates_bias:
                gate_out += self.gates_bias[idx]
            gate_out = nn.Softmax(dim=-1)(gate_out)
            gates_out.append(gate_out)

        outs = list()
        for gate_output in gates_out:
            expanded_gate_output = torch.unsqueeze(gate_output, 1) # batch * 1 * num_experts
            weighted_expert_output = experts_out * expanded_gate_output.expand_as(experts_out) # batch * mmoe_hidden_size * num_experts
            outs.append(torch.sum(weighted_expert_output, 2)) # batch * mmoe_hidden_size
          # task tower
        task_outputs = list()
        for i in range(self.num_task):
            oo = outs[i]
            mod=getattr(self, 'task_{}_dnn'.format(i+1))
            oo = mod(oo)
            task_outputs.append(oo)
        
        return task_outputs

class HighwayMLP(nn.Module):
    def __init__(self,
                 input_size,
                 gate_bias=-3,
                 activation_function=nn.functional.relu,
                 gate_activation=nn.functional.softmax):

        super(HighwayMLP, self).__init__()

        self.activation_function = activation_function
        self.gate_activation = gate_activation

        self.normal_layer = DNN(input_size,(input_size,input_size,input_size),dropout_rate=0.1)

        self.gate_layer = nn.Linear(input_size,input_size)

        self.gate_layer.bias.data.fill_(gate_bias)

    def forward(self, x):

        normal_layer_result = self.activation_function(self.normal_layer(x))
        gate_layer_result = self.gate_activation(self.gate_layer(x))

        multiplyed_gate_and_normal = torch.mul(normal_layer_result, gate_layer_result)
        multiplyed_gate_and_input = torch.mul((1 - gate_layer_result), x)

        return torch.add(multiplyed_gate_and_normal,
                         multiplyed_gate_and_input)
class Lookahead(Optimizer):
    def __init__(self, optimizer, k=5, alpha=0.5):
        self.optimizer = optimizer
        self.k = k
        self.alpha = alpha
        self.param_groups = self.optimizer.param_groups
        self.state = defaultdict(dict)
        self.fast_state = self.optimizer.state
        for group in self.param_groups:
            group["counter"] = 0

    def update(self, group):
        for fast in group["params"]:
            param_state = self.state[fast]
            if "slow_param" not in param_state:
                param_state["slow_param"] = torch.zeros_like(fast.data)
                param_state["slow_param"].copy_(fast.data)
            slow = param_state["slow_param"]
            slow += (fast.data - slow) * self.alpha
            fast.data.copy_(slow)

    def update_lookahead(self):
        for group in self.param_groups:
            self.update(group)

    def step(self, closure=None):
        loss = self.optimizer.step(closure)
        for group in self.param_groups:
            if group["counter"] == 0:
                self.update(group)
            group["counter"] += 1
            if group["counter"] >= self.k:
                group["counter"] = 0
        return loss

    def state_dict(self):
        fast_state_dict = self.optimizer.state_dict()
        slow_state = {(id(k) if isinstance(k, torch.Tensor) else k): v
            for k, v in self.state.items()
        }
        fast_state = fast_state_dict["state"]
        param_groups = fast_state_dict["param_groups"]
        return {
            "fast_state": fast_state,
            "slow_state": slow_state,
            "param_groups": param_groups,
        }

    def load_state_dict(self, state_dict):
        slow_state_dict = {
            "state": state_dict["slow_state"],
            "param_groups": state_dict["param_groups"],
        }
        fast_state_dict = {
            "state": state_dict["fast_state"],
            "param_groups": state_dict["param_groups"],
        }
        super(Lookahead, self).load_state_dict(slow_state_dict)
        self.optimizer.load_state_dict(fast_state_dict)
        self.fast_state = self.optimizer.state

    def add_param_group(self, param_group):
        param_group["counter"] = 0
        self.optimizer.add_param_group(param_group)

from torch.optim.lr_scheduler import LambdaLR
class WarmupLinearSchedule(LambdaLR):
    """ Linear warmup and then linear decay.
        Multiplies the learning rate defined in the optimizer by a dynamic variable determined by the current step.
        Linearly increases the multiplicative variable from 0. to 1. over `warmup_steps` training steps.
        Linearly decreases the multiplicative variable from 1. to 0. over remaining `t_total - warmup_steps` steps.
    """
    def __init__(self, optimizer, warmup_steps, t_total, last_epoch=-1):
        self.warmup_steps = warmup_steps
        self.t_total = t_total
        super(WarmupLinearSchedule, self).__init__(optimizer, self.lr_lambda, last_epoch=last_epoch)

    def lr_lambda(self, step):
        if step < self.warmup_steps:
            return float(step) / float(max(1, self.warmup_steps))
        return max(0.0, float(self.t_total - step) / float(max(1.0, self.t_total - self.warmup_steps)))
class Attn(nn.Module):
    def __init__(self,hidden_size):
        super(Attn, self).__init__()
        self.attn = nn.Linear(hidden_size,1)
    def forward(self, x):
        '''
        :param hidden: 
            previous hidden state of the decoder, in shape (layers*directions,B,H)
        :param encoder_outputs:
            encoder outputs from Encoder, in shape (T,B,H)
        :param src_len:
            used for masking. NoneType or tensor in shape (B) indicating sequence length
        :return
            attention energies in shape (B,T)
        '''   
        att=self.attn(x)
        att=F.tanh(att)
        att=F.softmax(att,1)
        att_x=att*x
        return att_x.sum(1)   
class AdamW(Optimizer):
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-6, weight_decay=0.0, correct_bias=True):
        if lr < 0.0:
            raise ValueError("Invalid learning rate: {} - should be >= 0.0".format(lr))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter: {} - should be in [0.0, 1.0[".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter: {} - should be in [0.0, 1.0[".format(betas[1]))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {} - should be >= 0.0".format(eps))
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay,
                        correct_bias=correct_bias)
        super(AdamW, self).__init__(params, defaults)

    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError('Adam does not support sparse gradients, please consider SparseAdam instead')
                state = self.state[p]
                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p.data)
                    state['exp_avg_sq'] = torch.zeros_like(p.data)
                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']
                state['step'] += 1
                exp_avg.mul_(beta1).add_(1.0 - beta1, grad)
                exp_avg_sq.mul_(beta2).addcmul_(1.0 - beta2, grad, grad)
                denom = exp_avg_sq.sqrt().add_(group['eps'])
                step_size = group['lr']
                if group['correct_bias']:  # No bias correction for Bert
                    bias_correction1 = 1.0 - beta1 ** state['step']
                    bias_correction2 = 1.0 - beta2 ** state['step']
                    step_size = step_size * math.sqrt(bias_correction2) / bias_correction1
                p.data.addcdiv_(-step_size, exp_avg, denom)
                if group['weight_decay'] > 0.0:
                    p.data.add_(-group['lr'] * group['weight_decay'], p.data)
        return loss
    
def build_optimizer(model, train_steps, learning_rate):
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, correct_bias=False, eps=1e-8)
    optimizer = Lookahead(optimizer, 5, 1)
    scheduler = WarmupLinearSchedule(optimizer, warmup_steps=train_steps * 0.1, t_total=train_steps)
    return optimizer, scheduler

def n_evaluate_nn(val_df,action_list,batch_size=512):
    model.eval()
    leng=len(val_df)
    val_src=val_df['userid'].apply(lambda x:userid2nid[x]).tolist()
    val_dst=val_df['feedid'].apply(lambda x:feedid2nid[x]).tolist()
    val_dense=torch.from_numpy(val_df[feat].values).float()
#     regs=torch.from_numpy(train_ratings['reg'].values).float()
    val_pred=[]
    all_aucs=[]
    weights=[0.30769231, 0.23076923, 0.15384615, 0.07692308, 0.07692308,0.07692308, 0.07692308]
    with torch.no_grad():
        for i in tqdm(range(0,leng//batch_size+1)):
            #         print(i*batch_size,(i+1)*batch_size)
            batch_src=val_src[i*batch_size:(i+1)*batch_size]
            batch_dst=val_dst[i*batch_size:(i+1)*batch_size]
            batch_dense=val_dense[i*batch_size:(i+1)*batch_size].to(torch.device('cuda'))
            pred=model(batch_src,batch_dst,batch_dense)
            val_pred.append(torch.cat(pred,axis=-1).sigmoid().cpu().numpy())
        val_pred=np.concatenate(val_pred,axis=0)
        for i,action in enumerate(action_list):
            val_df['pred_'+action]=val_pred[:,i]
            label_nunique = val_df.groupby(by='userid')[action].transform('nunique')
            tmp_df = val_df[label_nunique == 2]
            aucs = tmp_df.groupby(by='userid').apply(
                lambda x: roc_auc_score(x[action].values, x['pred_'+action].values))
            all_aucs.append(np.mean(aucs))
            print('val %s uauc:'%action,np.mean(aucs))
            print('val %s auc:'%action,roc_auc_score(val_df[action].values,val_pred[:,i]))
        print('score uauc:',sum([all_aucs[i]*weights[i] for i in range(len(action_list))]))
def evaluate_nn(val_df,action,batch_size=512):
    model.eval()
    leng=len(val_df)
    val_src=val_df['userid'].apply(lambda x:userid2nid[x]).tolist()
    val_dst=val_df['feedid'].apply(lambda x:feedid2nid[x]).tolist()
    val_pred=[]
    with torch.no_grad():
        for i in tqdm(range(0,leng//batch_size+1)):
            #         print(i*batch_size,(i+1)*batch_size)
            batch_src=val_src[i*batch_size:(i+1)*batch_size]
            batch_dst=val_dst[i*batch_size:(i+1)*batch_size]

            pred=model(batch_src,batch_dst)

            val_pred.append(pred.sigmoid().view(-1).cpu().numpy())
        val_pred=np.concatenate(val_pred,axis=-1)
        val_df['pred_'+action]=val_pred
        label_nunique = val_df.groupby(by='userid')[action].transform('nunique')
        tmp_df = val_df[label_nunique == 2]
        
        aucs = tmp_df.groupby(by='userid').apply(
            lambda x: roc_auc_score(x[action].values, x['pred_'+action].values))
        print('val uauc:',np.mean(aucs))
        print('val auc:',roc_auc_score(val_df[action].values,val_pred))

In [33]:
for f,d in user_data.items():
    user_data[f]=d.to(torch.device('cuda'))
    
for f,d in feed_data.items():
    feed_data[f]=d.to(torch.device('cuda'))

In [34]:
def test_pred_func(model): 
    test_src=test_a['userid'].apply(lambda x:userid2nid[x]).tolist()
    test_dst=test_a['feedid'].apply(lambda x:feedid2nid[x]).tolist()
    batch_size=868
    test_dense=torch.from_numpy(test_a[feat].values).float()
    test_pred=[]
    model.eval()
    with torch.no_grad():
        for i in tqdm(range(0,len(test_a)//batch_size+1)):
    #         print(i*batch_size,(i+1)*batch_size)
            batch_src=test_src[i*batch_size:(i+1)*batch_size]
            batch_dst=test_dst[i*batch_size:(i+1)*batch_size]
            batch_dense=test_dense[i*batch_size:(i+1)*batch_size].cuda()
            pred=model(batch_src,batch_dst,batch_dense)
            pred=torch.cat(pred,axis=-1)
            test_pred.append(pred.sigmoid().cpu().numpy())
    test_pred=np.concatenate(test_pred,axis=0)
    return test_pred

In [37]:
import warnings
warnings.filterwarnings("ignore")

In [37]:
#del trn_dense,model,batch_dense,batch_dst,user_data,feed_data,optimizer
# del model
torch.cuda.empty_cache()
gc.collect()

1710

In [26]:
max_day=15

In [35]:
train_ratings=df[(df.date_<max_day)]
test_a=df[df.date_==15]
# test_a=test_a.reset_index(drop=True)
# val_ratings=df[df.date_==max_day]
# (~val_ratings.feedid.isin(train_ratings.feedid)).sum()/val_ratings.shape[0]
gc.collect()

0

In [36]:
graph_emb=np.concatenate([np.load('./tmp/grap_embedding32_sg2.npy'),np.load('./tmp/grap_embedding32_hs2.npy')],axis=1)
#graph_user=np.load('./tmp/grap_embedding_user.mpy.npy')

In [ ]:
## batch_size=4096
epochs=2
trn_dense=torch.from_numpy(train_ratings[feat].values).float()
model = Model(user_data,feed_data
             ,feed_embed=feed_emb,graph_emb=graph_emb) #in_features, hidden_features, out_features, rel_names


model=model.to(torch.device('cuda'))
# model = nn.DataParallel(model)

train_steps = int(len(train_ratings) * epochs / batch_size) + 1
optimizer, scheduler = build_optimizer(model, train_steps, learning_rate=1e-3)
all_pred=[]
src=train_ratings['userid'].apply(lambda x: userid2nid[x]).values
dst=train_ratings['feedid'].apply(lambda x: feedid2nid[x]).values
labels=torch.from_numpy(train_ratings[PREDICT_LIST].values).float()
regs=torch.from_numpy(train_ratings['reg'].values).float()
criti=nn.BCEWithLogitsLoss()
reg_criti=nn.MSELoss()
n_pos=len(train_ratings)
batch_index=np.arange(n_pos) # 生成正样本的index
for epoch in range(epochs):
    print('epoch: ----%d--'%epoch)
    random.shuffle(batch_index) 
    epoch_loss=0
    model.train()
    for ind in tqdm(range(0,n_pos//batch_size+1)):
        batch=batch_index[ind*batch_size:(ind+1)*batch_size]
        batch_dense=trn_dense[batch].to(torch.device('cuda'))
        batch_src=src[batch]
        batch_dst=dst[batch]
        
#         print(batch_src)
        logits = model(batch_src,batch_dst,batch_dense)
        batch_label=labels[batch].cuda()
        batch_reg=regs[batch].cuda()
        loss=criti(logits[0][:,0],batch_label[:,0])*0.8+criti(logits[1][:,0],batch_label[:,1])*0.8+\
        criti(logits[2][:,0],batch_label[:,2])*0.4+criti(logits[3][:,0],batch_label[:,3])*0.4+reg_criti(logits[7][:,0],batch_reg)*0.6+\
        criti(logits[4][:,0],batch_label[:,4])*0.3+criti(logits[5][:,0],batch_label[:,5])*0.3+criti(logits[6][:,0],batch_label[:,6])*0.3
        epoch_loss+=loss.item()
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
        scheduler.step()

        if ind%1000==0:
            print('binary loss:',loss.item())
            batch_label=batch_label.cpu().numpy()
            pred=torch.cat(logits,axis=-1).sigmoid().detach().cpu().numpy()
#             pred=logits.sigmoid().detach().cpu().numpy()
            for ii,aa in enumerate(PREDICT_LIST):
                try:
                    print('train %s auc:'%aa,roc_auc_score(batch_label[:,ii],pred[:,ii]))
                except:
                    continue
#             print(pred[:10])
        #if (epoch>0) and (ind in [500,1000,1608]):
    print('epoch %d  loss: %f '%(epoch,epoch_loss/(len(batch_index)//batch_size+1)))
#     n_evaluate_nn(val_df=val_ratings,action_list=PREDICT_LIST,batch_size=2048)
    
torch.save(model, './model_weight/my_deep_v2_v1.pth')

In [ ]:
0.642
0.633
0.716
0.714

0.661

In [ ]:
torch.save(model, './model_weight/my_deep_v2_v1.pth')

In [44]:
n_evaluate_nn(val_df=val_ratings,action_list=PREDICT_LIST,batch_size=2048)

100%|██████████| 2981/2981 [03:46<00:00, 13.18it/s]


val read_comment uauc: 0.6600392946556747
val read_comment auc: 0.9344231687418318
val like uauc: 0.6461100589338815
val like auc: 0.8506679951060663
val click_avatar uauc: 0.7446228446122458
val click_avatar auc: 0.866149472317661
val forward uauc: 0.7379559037865497
val forward auc: 0.8913635371361742
val comment uauc: 0.6247119469234252
val comment auc: 0.9011526478701319
val follow uauc: 0.732700484385476
val follow auc: 0.8841244611561798
val favorite uauc: 0.7723210439695183
val favorite auc: 0.9440381597709615
score uauc: 0.687340193425409


In [38]:
test_pred=test_pred_func(model)

100%|██████████| 4899/4899 [06:22<00:00, 12.82it/s]


In [39]:
test_a[PREDICT_LIST]=test_pred[:,:7]

/home/tione/notebook/envs/tf1/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/tione/notebook/envs/tf1/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [40]:
sub=test_a[['userid','feedid']+PREDICT_LIST]

In [45]:
sub.to_csv('./upload/deep_v2_v1_sample.csv',index=False)

In [42]:
test_a=pd.read_csv('./wbdc2021/data/wedata/wechat_algo_data2/test_a.csv')

In [47]:
test_a=df[df.date_==15]

In [4]:
PREDICT_LIST=["read_comment","like", "click_avatar", "forward"]

In [46]:
sub1=pd.read_csv('./upload/deep_v2_v1_sample.csv')
sub2=pd.read_csv('./upload/deep_v2+v1.csv')
# sub3=pd.read_csv('./my_deep_v2_4.csv')

In [47]:
test_pred=sub1[PREDICT_LIST].values*0.5+sub2[PREDICT_LIST].values*0.5

In [48]:
test_a=pd.read_csv('./wbdc2021/data/wedata/wechat_algo_data2/test_a.csv')

In [52]:
sub=test_a[['userid','feedid']]
for i in range(len(PREDICT_LIST)):
    sub[PREDICT_LIST[i]]=test_pred[:,i]

In [53]:
sub.to_csv('./upload/ensenmbel.csv',index=False)

In [54]:
sub

,userid,feedid,read_comment,like,click_avatar,forward,comment,follow,favorite
0,175282,50458,0.019492,0.004499,0.000405,0.017588,0.000049,0.000006,0.000058
1,80036,42329,0.006714,0.004977,0.025233,0.000853,0.000035,0.001474,0.000175
2,145791,85242,0.000101,0.002875,0.001582,0.000742,0.000005,0.000079,0.000088
3,28430,9425,0.000026,0.005486,0.062523,0.050482,0.000274,0.004173,0.000052
4,44393,11866,0.002365,0.000963,0.001640,0.000940,0.000001,0.000018,0.000022
...,...,...,...,...,...,...,...,...,...
4252092,153322,51633,0.001272,0.005847,0.004927,0.007423,0.000004,0.000134,0.000120
4252093,39430,20147,0.003737,0.008553,0.000178,0.000141,0.000032,0.000014,0.000017
4252094,2524,89043,0.001360,0.008568,0.001226,0.000761,0.000005,0.000050,0.000231
4252095,69629,27238,0.028333,0.004441,0.020382,0.000261,0.000050,0.000820,0.001176


In [52]:
sub['read_comment']=(sub1['read_comment']+sub2['read_comment']).values/2
sub['like']=(sub1['like']+sub2['like']).values/2
sub['click_avatar']=(sub1['click_avatar']+sub2['click_avatar']).values/2
sub['forward']=(sub2['forward']+sub1['forward']).values/2

<ipython-input-52-bc76750da07e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub['read_comment']=(sub1['read_comment']+sub2['read_comment']).values/2
<ipython-input-52-bc76750da07e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub['like']=(sub1['like']+sub2['like']).values/2
<ipython-input-52-bc76750da07e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

In [53]:
sub.to_csv('my_deep_ensemble_21.csv',index=False)

In [1]:
import pandas as pd


In [2]:
sub=pd.read_csv('./upload/deep_v2+v1.csv')

In [3]:
sub.head()

,userid,feedid,read_comment,like,click_avatar,forward,comment,follow,favorite
0,175282,50458,0.013006,0.004985,0.000409,0.019743,0.000052,0.000007,0.000011
1,80036,42329,0.004962,0.003664,0.026879,0.001048,0.000009,0.001628,0.000312
2,145791,85242,0.000127,0.002968,0.001970,0.000237,0.000005,0.000068,0.000078
3,28430,9425,0.000049,0.002507,0.069001,0.055966,0.000152,0.003369,0.000100
4,44393,11866,0.002277,0.000740,0.001726,0.001206,0.000001,0.000031,0.000041


In [4]:
test_a=pd.read_csv('./wbdc2021/data/wedata/wechat_algo_data2/test_a.csv')